In [1]:
import torch
from nncf import NNCFConfig
from nncf.common.logging import nncf_logger
from nncf.torch import create_compressed_model, register_default_init_args
from nncf.torch.initialization import PTInitializingDataLoader
from nncf.torch.layer_utils import CompressionParameter

from PIL import Image
import open_clip

/home/alex/virt_envs/tomeov/lib/python3.8/site-packages/openvino/offline_transformations/__init__.py:10: FutureWarning: The module is private and following namespace `offline_transformations` will be removed in the future.
  warnings.warn(


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [2]:
model, train_transform, eval_transform = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained='laion400m_e32')

In [3]:
tokenizer = open_clip.get_tokenizer('ViT-B-16-plus-240')

image = eval_transform(Image.open("../docs/CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

### Prepare data for optimization

In [16]:
import os
import random
from io import BytesIO
import requests
import numpy as np

def get_pil_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    return image.convert("RGB")

BACKUP_PAIR = (
    get_pil_from_url(
        "https://thumbs.dreamstime.com/t/altai-mountains-mountain-lake-russia-siberia-chuya-ridge-49130812.jpg"
    ),
    "Altai mountains Stock Photography",
)
AVAILABLE_EXAMPLES = []

def check_text_data(data):
    if isinstance(data, str):
        return True
    if isinstance(data, list):
        return all(isinstance(x, str) for x in data)
    return False    

def laion2B_preprocess_train(examples, train_transforms, tokenize_captions, image_column="url", text_column="caption"):
    url = examples[image_column]
    try:
        image = get_pil_from_url(url)
        if not check_text_data(examples[text_column]):
            raise ValueError("Text data is not valid")
        AVAILABLE_EXAMPLES.append((url, examples[text_column]))
    except Exception:
        print(f"Can't load image from url: {url}, using cache with size: {len(AVAILABLE_EXAMPLES)}")
        if len(AVAILABLE_EXAMPLES) > 0:
            backup_id = random.randint(0, len(AVAILABLE_EXAMPLES) - 1)
            backup_example = AVAILABLE_EXAMPLES[backup_id]
            try:
                image = get_pil_from_url(backup_example[0])
                examples[text_column] = backup_example[1]
            except Exception:
                print(f"Can't load image from cached url: {backup_example[0]}, using backup")
                image = BACKUP_PAIR[0].copy()
                examples[text_column] = BACKUP_PAIR[1]
        else:
            print(f"Can't load image from url: {url}, using backup")
            image = BACKUP_PAIR[0].copy()
            examples[text_column] = BACKUP_PAIR[1]

    examples["pixel_values"] = train_transforms(image)
    examples["input_ids"] = tokenize_captions(examples)
    return examples

def tokenize_captions(examples, is_train=True):
    caption_column = "caption"
    captions = []
    caption = examples[caption_column]
    if isinstance(caption, str):
        captions.append(caption)
    elif isinstance(caption, (list, np.ndarray)):
        # take a random caption if there are multiple
        captions.append(random.choice(caption) if is_train else caption[0])
    else:
        raise ValueError(f"Caption column `{caption_column}` should contain either strings or lists of strings.")
    #inputs = tokenizer(captions[0], max_length=tokenizer.model_max_length, padding="do_not_pad", truncation=True)
    #input_ids = inputs.input_ids
    input_ids = tokenizer(captions[0])[0]
    return input_ids

In [17]:
from datasets import load_dataset

max_train_samples = 10000
dataset = load_dataset("laion/laion400m", streaming=True)
train_dataset = dataset["train"].shuffle(seed=42, buffer_size=max_train_samples)

In [18]:
def collate_fn_image(examples):
    examples = [laion2B_preprocess_train(example, train_transform, tokenize_captions) for example in examples]
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    input_ids = torch.stack([example["input_ids"] for example in examples])
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        # "attention_mask": padded_tokens.attention_mask,
    }

In [19]:
import itertools
from tqdm.notebook import tqdm

def prepare_nncf_init_data(dataloader, init_steps):
    nncf_init_data = []

    print(f"Fetching {init_steps} for the initialization...")
    for _, batch in tqdm(zip(range(init_steps), itertools.islice(dataloader, 0, init_steps))):
        with torch.no_grad():
            # Convert images to latent space
            
            nncf_init_data.append(
                (
                    batch["pixel_values"].to("cpu"),
                    batch["input_ids"].to("cpu")
                )
            )
    return nncf_init_data

In [20]:
train_batch_size = 1
dataloader_num_workers = 4
train_dataloader = torch.utils.data.DataLoader(
        train_dataset, collate_fn=collate_fn_image, batch_size=train_batch_size, num_workers=dataloader_num_workers
    )

In [21]:
opt_init_steps = 100
init_data = prepare_nncf_init_data(train_dataloader, opt_init_steps)

Fetching 100 for the initialization...


0it [00:00, ?it/s]

Can't load image from url: https://s3.amazonaws.com/whataspace/space_pictures/pictures/000/033/180/fullwidth/_DSC0146.jpg?1591610094, using cache with size: 0
Can't load image from url: https://s3.amazonaws.com/whataspace/space_pictures/pictures/000/033/180/fullwidth/_DSC0146.jpg?1591610094, using backup
Can't load image from url: https://i0.wp.com/des.gearbest.com/uploads/pdm-desc-pic/Electronic/image/2016/11/25/1480056293210706.jpg?w=960, using cache with size: 2
Can't load image from url: https://static.paraforme.fr/images/products/juvamine-concentre-chrome.jpg, using cache with size: 2
Can't load image from url: http://cdn3.static-homes.com/cgi-bin/readimage/9eb0ae4fa92b5dfd09b03c9e3dc997c5_1_resizeto_193x143x1, using cache with size: 3
Can't load image from url: https://img.shellporn.com/spcs/thumbs/155/299_hotel_wang_.jpg, using cache with size: 5
Can't load image from url: https://sslh.ulximg.com/image/740x493/cover/1533483761_ac8b9fbc89ca57a7269ec0c7f5947094.jpg/2143af354a3c53d

In [22]:
class InitDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.init_data = data

    def __len__(self):
        return len(self.init_data)

    def __getitem__(self, index):
        return self.init_data[index]

### Apply NNCF optimizations

In [23]:
image_encoder_config = {
    "input_info": [
        {  
            "sample_size": [1, 3, 240, 240]
        },
    ],
    "log_dir": "./",  # The log directory for NNCF-specific logging outputs.
    "compression": [
        {
            "algorithm": "quantization",  # Specify the algorithm here.
            "preset": "mixed",
            "initializer": {
                "range": {"num_init_samples": opt_init_steps},
                "batchnorm_adaptation": {"num_bn_adaptation_samples": opt_init_steps},
            },
            "scope_overrides": {"activations": {"{re}.*baddbmm_0": {"mode": "symmetric"}}},
            "ignored_scopes": [
                "{re}.*__add___[0-2]",
                "{re}.*layer_norm_0",
                "{re}.*Attention.*/bmm_0",
                "{re}.*__truediv__*",
                "{re}.*group_norm_0",
                "{re}.*mul___[0-2]",
                "{re}.*silu_[0-2]",
            ],
            "export_to_onnx_standard_ops": True,
        },
    ],
}

init_dataloader = torch.utils.data.DataLoader(InitDataset(init_data), batch_size=1, num_workers=1)

class UnetInitDataLoader(PTInitializingDataLoader):
    def get_inputs(self, dataloader_output):
        image = dataloader_output["pixel_values"].float().to(model.device, non_blocking=True)
        return (image), {}

    def get_target(self, dataloader_output):
        return dataloader_output["pixel_values"]

nncf_config = NNCFConfig.from_dict(image_encoder_config)
nncf_config = register_default_init_args(nncf_config, UnetInitDataLoader(init_dataloader))

In [25]:
compression_controller, image_encoder = create_compressed_model(model.visual, nncf_config)

AttributeError: 'NoneType' object has no attribute 'shape'